In [1]:
import numpy as np
import pandas as pd
import re
from nltk.tokenize import TreebankWordTokenizer

import matplotlib
import sklearn
from IPython.core.display import display, HTML

######################################################################################################################

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

get_ipython().run_line_magic('matplotlib', 'inline')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid", font_scale=1.9)

######################################################################################################################

# TO WORK WITH
import pandas as pd
import numpy as np
from numpy import set_printoptions

# HIDE WARNINGS
import warnings
warnings.filterwarnings('ignore')

# PREPROCESSING & MODEL SELECTION
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from scipy.stats import randint
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression, LinearRegression, RidgeCV, LassoCV, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.metrics import SCORERS
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

# print(SCORERS.keys())

# PLOTTING
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("notebook")
get_ipython().run_line_magic('matplotlib', 'inline')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sklearn import tree
from graphviz import Source
from matplotlib.pylab import rcParams
import matplotlib.lines as mlines
from yellowbrick.classifier import PrecisionRecallCurve
from yellowbrick.classifier import ROCAUC
from yellowbrick.classifier import ConfusionMatrix
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.cluster import SilhouetteVisualizer
from yellowbrick.cluster import InterclusterDistance
import plotly.express as px
import scipy.cluster.hierarchy as sch
from sklearn.metrics import classification_report


# STANDARD MODELS
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# ENSEMBLE
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

# XGBOOST
from xgboost import XGBClassifier
import xgboost as xgb

# CLUSTERING
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

# PICKLE
from pickle import dump
from pickle import load
from sklearn.metrics import mean_squared_error




In [2]:
def split(data):  
    '''
    Clean the data and return an array for the target variable and all the input variables

    '''    
    data=data.dropna()
    
    data=data[data["price_per_night"]>=12]
    data=data[data["number_of_reviews"]>=1]
    data=data[data["guests_included"]>=1]
    
    

    data['extra_price']=data['security_deposit']+data['cleaning_fee']+data['extra_people']
    data['extra_price'].describe()
    
    
    for test in data['extra_price']:
       
        if test<25:
            data['extra_price']=25
    
    
        if 25<test and test<100:
            data['extra_price']=75
    
    
        if 100<test and test<235:
            data['extra_price']=125
    
    
        if 235<test:
            data['extra_price']=235
            
    
    for categorical_feature in ['neighbourhood_cleansed',"property_type","room_type"]:
        data = pd.concat([data,pd.get_dummies(data[categorical_feature], prefix=categorical_feature, prefix_sep='_',)], axis=1)
        
    
    data= data.drop(['neighbourhood_cleansed', 'property_type', 'room_type','description','host_about_bool',
                    'amenities',"house_rules","review_scores_cleanliness","review_scores_rating",'review_scores_accuracy',              
                    'review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_value',
                     'id',"property_type", "host_response_rate", 'reviews_per_month','number_of_reviews'
                     ,'security_deposit','cleaning_fee','extra_people','extra_price','availability_365'],axis=1)
    
    
    y = data["price_per_night"] # Target variable (price_per_night)
    
    X = data.drop(["price_per_night"],axis=1) 

    return (X,y)

In [3]:
def XGB(X,y):
    '''
    Calculating RMSE with XGBoost Hyperparameters

    '''
    from sklearn.model_selection import train_test_split

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

    xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.2,
                max_depth = 7, alpha = 20, n_estimators = 70)
    xg_reg.fit(X_train,y_train)

    preds = xg_reg.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    print("RMSE: %f" % (rmse))
    return

In [13]:
def LinReg(X,y):
    '''
    Performing Linear Regression with scikit-learn

    '''


    from sklearn.linear_model import LinearRegression       
                                                         
    kfold=KFold(n_splits=10, random_state=7)                
                                   
    model=LinearRegression()
    scoring = "neg_mean_squared_error"

    results=cross_val_score(model, X, y, cv=kfold, scoring=scoring)

    print(f'Linear Regression - MSE {results.mean():.3f} std {results.std():.3f}')


    # Now lets use it in the same way than the statsmodel

    model_x=LinearRegression()
    model_x.fit(X,y)
    print(f'Intercept {model_x.intercept_:.4f}')
    print("Coefficients ",model_x.coef_)

    y_pred_x=model_x.predict(X)
    
    regressor = LinearRegression()  
    regressor.fit(X, y)
    print("predict")
    y_pred = regressor.predict(X)
    df = pd.DataFrame({'Actual': y, 'Predicted': y_pred})
    df.head(25)
    print(df)


    print(f'MAE - Mean Absolute Error {metrics.mean_absolute_error(y, y_pred_x):.3f}')
    print(f'MSE - Mean Square Error  {metrics.mean_squared_error(y, y_pred_x):.3f}')
    print(f'R2    {metrics.r2_score(y, y_pred_x):.3f}')
    return

In [14]:
data = pd.read_csv("listings_8.csv")

X,y= split(data)

In [15]:
XGB(X,y)

[11:50:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
RMSE: 62.013981


In [16]:
LinReg(X,y)


Linear Regression - MSE -247339948278107662786047770624.000 std 742017984384899230796190056448.000
Intercept -42167051653739.7734
Coefficients  [-3.30923961e+00 -2.94685902e+00  7.83000964e+00 -3.95446917e+01
 -4.54969204e+01 -3.40970068e+01 -2.39027639e+01  1.18635114e+01
  7.36786495e-01 -4.11650556e-01  4.64321848e+00  2.39303359e+01
  4.93787630e+01  4.74837320e+00  6.69160690e+00  1.70138058e+01
  2.84804693e+00 -1.40903687e+01  1.34778252e+01 -1.41159208e+01
 -2.17052243e+12  9.39628472e+00 -1.19046522e+00  2.17052243e+12
  9.68525614e+00  5.77941984e+00 -5.84684273e+00  7.46324016e-01
  1.26432205e+01  1.64175684e+00  6.28915287e+00 -2.22660678e+01
  4.09343750e+00 -1.06646347e+01  2.77432861e+01 -6.06530376e+12
 -6.06530376e+12 -6.06530376e+12 -1.67700195e+01  2.32590332e+01
  1.27366058e+13  1.27366058e+13  1.27366058e+13  1.27366058e+13
  1.27366058e+13  1.27366058e+13  1.27366058e+13  1.27366058e+13
  1.27366058e+13  1.27366058e+13  1.27366058e+13  1.27366058e+13
  1.2736605

In [8]:
# data.info()



# df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
# df1 = df.head(25)